
# Using pre-trained models to solve "standard" NLP problems **without** fine-tuning

Adapted from https://github.com/nlp-with-transformers/notebooks/blob/main/01_introduction.ipynb

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### Text Classification

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
outputs = classifier(text)

In [ ]:
outputs

[{'label': 'NEGATIVE', 'score': 0.9015460014343262}]

In [ ]:
import pandas as pd
pd.DataFrame(outputs)    

,label,score
0,NEGATIVE,0.901546


Let's try another example.


In [ ]:
inputs = ['i hated the movie', 
          'if i said i loved the movie, i would be lying', 
          'the movie left me speechless', 
          ' incredible!'
          ]

In [ ]:
outputs = classifier(inputs)
pd.DataFrame(outputs)  

,label,score
0,NEGATIVE,0.999695
1,NEGATIVE,0.992098
2,NEGATIVE,0.991616
3,POSITIVE,0.999877


### Named Entity Recognition

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
outputs = ner_tagger(text)
pd.DataFrame(outputs)    

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


### Question Answering 

In [ ]:
reader = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])    

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


### Summarization

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
outputs = summarizer(text, max_length=56, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I


This is pretty bad 🙁. The above output is from the **default** model for summarization. Let's try the most popular summarization model from the HuggingFace Hub.

In [ ]:
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

In [ ]:
outputs = summarizer(text, max_length=56, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

Bumblebee ordered an Optimus Prime action figure from Amazon's online store in Germany, but was sent an action figure of Megatron instead. Bumblebee wants an exchange of the Megatron figure for the Optimus Prime figure he ordered. He sent copies of


Much better!

### Text Generation

In [ ]:
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

In [ ]:
generator = pipeline("text-generation")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response

In [ ]:
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. For one, I received a paper file from a store just outside your country and sent it to you by a customer whose name was not listed.

To remove any further records, however, I can make sure I can send a copy of my purchase to you via USPS.

Any questions or comments?

I need


Yikes!! Let's try a bigger model.

In [ ]:
generator = pipeline("text-generation", model='bigscience/bloom-560m')

In [ ]:
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. Please contact us to discuss the issue. We will be happy to help you. Thank you for your understanding.

A:

I have been using the Optimus Prime for a while now and I have never had any problems with it. I have been using it for a couple of years now and


Hmm ... 🤔